In [8]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import accuracy_score
import os
import scipy.io
import math

from sklearn.utils import shuffle

from PIL import Image
import requests
from io import BytesIO

from tensorflow.keras.applications import resnet50
from keras.preprocessing import image

import matplotlib.pyplot as plt

import numpy as np

from keras.applications.imagenet_utils import preprocess_input, decode_predictions

import tensorflow as tf
from keras.preprocessing import image

from sklearn.model_selection import train_test_split

from scipy import spatial
from tqdm import tqdm

import gc

In [9]:
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from scipy import spatial
import gc
import scipy.io
from PIL import Image

In [10]:
gc.collect()

48

In [11]:
styles = pd.read_csv('.\\fashion-dataset\\styles.csv', error_bad_lines=False)
print(styles.head())

      id gender masterCategory subCategory  articleType baseColour  season  \
0  15970    Men        Apparel     Topwear       Shirts  Navy Blue    Fall   
1  39386    Men        Apparel  Bottomwear        Jeans       Blue  Summer   
2  59263  Women    Accessories     Watches      Watches     Silver  Winter   
3  21379    Men        Apparel  Bottomwear  Track Pants      Black    Fall   
4  53759    Men        Apparel     Topwear      Tshirts       Grey  Summer   

     year   usage                             productDisplayName  
0  2011.0  Casual               Turtle Check Men Navy Blue Shirt  
1  2012.0  Casual             Peter England Men Party Blue Jeans  
2  2016.0  Casual                       Titan Women Silver Watch  
3  2011.0  Casual  Manchester United Men Solid Black Track Pants  
4  2012.0  Casual                          Puma Men Grey T-shirt  


b'Skipping line 6044: expected 10 fields, saw 11\nSkipping line 6569: expected 10 fields, saw 11\nSkipping line 7399: expected 10 fields, saw 11\nSkipping line 7939: expected 10 fields, saw 11\nSkipping line 9026: expected 10 fields, saw 11\nSkipping line 10264: expected 10 fields, saw 11\nSkipping line 10427: expected 10 fields, saw 11\nSkipping line 10905: expected 10 fields, saw 11\nSkipping line 11373: expected 10 fields, saw 11\nSkipping line 11945: expected 10 fields, saw 11\nSkipping line 14112: expected 10 fields, saw 11\nSkipping line 14532: expected 10 fields, saw 11\nSkipping line 15076: expected 10 fields, saw 12\nSkipping line 29906: expected 10 fields, saw 11\nSkipping line 31625: expected 10 fields, saw 11\nSkipping line 33020: expected 10 fields, saw 11\nSkipping line 35748: expected 10 fields, saw 11\nSkipping line 35962: expected 10 fields, saw 11\nSkipping line 37770: expected 10 fields, saw 11\nSkipping line 38105: expected 10 fields, saw 11\nSkipping line 38275: ex

In [12]:
resnet50_model = resnet50.ResNet50(weights='imagenet')

In [13]:
colors = styles['baseColour'].unique()
colors

array(['Navy Blue', 'Blue', 'Silver', 'Black', 'Grey', 'Green', 'Purple',
       'White', 'Beige', 'Brown', 'Bronze', 'Teal', 'Copper', 'Pink',
       'Off White', 'Maroon', 'Red', 'Khaki', 'Orange', 'Coffee Brown',
       'Yellow', 'Charcoal', 'Gold', 'Steel', 'Tan', 'Multi', 'Magenta',
       'Lavender', 'Sea Green', 'Cream', 'Peach', 'Olive', 'Skin',
       'Burgundy', 'Grey Melange', 'Rust', 'Rose', 'Lime Green', 'Mauve',
       'Turquoise Blue', 'Metallic', 'Mustard', 'Taupe', 'Nude',
       'Mushroom Brown', nan, 'Fluorescent Green'], dtype=object)

In [18]:
# shirts_navyblue = styles[(styles['articleType'] == 'Shirts') & (styles['baseColour'] == 'Navy Blue')]['id'].to_numpy()
# shirts_blue = styles[(styles['articleType'] == 'Shirts') & (styles['baseColour'] == 'Blue')]['id'].to_numpy()
# shirts_black = styles[(styles['articleType'] == 'Shirts') & (styles['baseColour'] == 'Black')]['id'].to_numpy()
# shirts_grey = styles[(styles['articleType'] == 'Shirts') & (styles['baseColour'] == 'Grey')]['id'].to_numpy()
# shirts_green = styles[(styles['articleType'] == 'Shirts') & (styles['baseColour'] == 'Green')]['id'].to_numpy()
# shirts_purple = styles[(styles['articleType'] == 'Shirts') & (styles['baseColour'] == 'Purple')]['id'].to_numpy()
# shirts_white = styles[(styles['articleType'] == 'Shirts') & (styles['baseColour'] == 'White')]['id'].to_numpy()
# shirts = styles[(styles['articleType'] == 'Shirts') & (styles['baseColour'] == 'Beige')]

articles = []
for i, color in enumerate(colors):    
    for article in ['Tshirts',
                    'Jeans',
                    'Sports Shoes']:
        x = styles[(styles['articleType'] == article) & (styles['baseColour'] == color)].shape[0]
        if x >= 100:
            if (article == 'Tshirts'):
                tshirts = styles[(styles['articleType'] == article) & (styles['baseColour'] == color)]['id'].to_numpy()
                print(tshirts.head(100).shape)
                continue
            articles.append(styles[(styles['articleType'] == article) & (styles['baseColour'] == color)]['id'].to_numpy())
            print(article, color)
print(len(articles))
print(sum(map(len, articles)))

# shirts_navyblue.shape, shirts_blue.shape, shirts_black.shape, shirts_grey.shape, shirts_green.shape, shirts_purple.shape, shirts_white.shape

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [11]:
image_path = '.\\fashion-dataset\\images\\'

IMG_SIZE = 224
LIMIT_IMAGES = 196
NUM_OUTPUTS = 3

In [12]:
def load_imgs(names):
    imgs = []
    for i, image_name in enumerate(tqdm(names)):
#         if i% 50 == 0 :
#             print(f"Loading Image {i}")
        try:
            img = image.load_img(f'{image_path}{image_name}.jpg', target_size=(IMG_SIZE, IMG_SIZE))
        except:
            img = None
        if img is None:
            continue
        img = np.array(img)
        imgs.append(img)
    return np.array(imgs)

In [13]:
# print("Loading...")
# shirts_navyblue_images = load_imgs(shirts_navyblue[:LIMIT_IMAGES])
# shirts_blue_images = load_imgs(shirts_blue[:LIMIT_IMAGES])
# shirts_black_images = load_imgs(shirts_black[:LIMIT_IMAGES])
# shirts_grey_images = load_imgs(shirts_grey[:LIMIT_IMAGES])
# shirts_green_images = load_imgs(shirts_green[:LIMIT_IMAGES])
# shirts_purple_images = load_imgs(shirts_purple[:LIMIT_IMAGES])
# shirts_white_images = load_imgs(shirts_white[:LIMIT_IMAGES])
# shirts_navyblue_images.shape, shirts_blue_images.shape, shirts_black_images.shape, shirts_grey_images.shape, shirts_green_images.shape, shirts_purple_images.shape, shirts_white_images.shape

In [14]:
# print("Loading...")
# shirts_navyblue_images = load_imgs(shirts_navyblue)
# shirts_blue_images = load_imgs(shirts_blue)
# shirts_black_images = load_imgs(shirts_black)
# shirts_grey_images = load_imgs(shirts_grey)
# shirts_green_images = load_imgs(shirts_green)
# shirts_purple_images = load_imgs(shirts_purple)
# shirts_white_images = load_imgs(shirts_white)
# shirts_navyblue_images.shape, shirts_blue_images.shape, shirts_black_images.shape, shirts_grey_images.shape, shirts_green_images.shape, shirts_purple_images.shape, shirts_white_images.shape

In [ ]:
new_articles = []
for i, article in enumerate(articles):
    new_articles.append(load_imgs(article))
    print(i)

  0%|                                                                              | 1/1046 [00:00<02:21,  7.40it/s]

0


  0%|                                                                                       | 0/446 [00:00<?, ?it/s]

1


 45%|██████████████████████████████████▌                                          | 200/446 [00:25<00:25,  9.47it/s]

In [ ]:
train_articles = []
test_articles = []

for new_article in new_articles:
    train_article, test_article, _, _ = train_test_split(new_article, np.repeat(0, new_article.shape[0]), test_size=0.2)
    train_articles.append(train_article)
    test_articles.append(test_article)

In [14]:
# train_shirts = []
# test_shirts = []

# for shirt in shirts:
#     train_shirt, test_shirt, _, _ = train_test_split(shirt, np.repeat(0, shirt.shape[0]), test_size=0.2)
#     train_shirts.append(train_shirt)
#     test_shirts.append(test_shirt)

In [15]:
# train_shirts_navyblue_images, test_shirts_navyblue_images, _, _ = train_test_split(shirts_navyblue_images, np.repeat(0, shirts_navyblue_images.shape[0]), test_size = 0.2)
# train_shirts_blue_images, test_shirts_blue_images, _, _ = train_test_split(shirts_blue_images, np.repeat(0, shirts_blue_images.shape[0]), test_size = 0.2)
# train_shirts_black_images, test_shirts_black_images, _, _ = train_test_split(shirts_black_images, np.repeat(0, shirts_black_images.shape[0]), test_size = 0.2)
# train_shirts_grey_images, test_shirts_grey_images, _, _ = train_test_split(shirts_grey_images, np.repeat(0, shirts_grey_images.shape[0]), test_size = 0.2)
# train_shirts_green_images, test_shirts_green_images, _, _ = train_test_split(shirts_green_images, np.repeat(0, shirts_green_images.shape[0]), test_size = 0.2)
# train_shirts_purple_images, test_shirts_purple_images, _, _ = train_test_split(shirts_purple_images, np.repeat(0, shirts_purple_images.shape[0]), test_size = 0.2)
# train_shirts_white_images, test_shirts_white_images, _, _ = train_test_split(shirts_white_images, np.repeat(0, shirts_white_images.shape[0]), test_size = 0.2)

In [ ]:
def get_vectors(imgs):
    processed_batch = preprocess_input(imgs, mode="caffe")
    return resnet50_model.predict(processed_batch)

def get_average_vector(imgs):
    vectors = get_vectors(imgs)
    print(vectors.shape)
    return np.mean(vectors, axis=0)

def closeness(a, b):
#     print(a.shape)
#     print(b.shape)
    return 1 - spatial.distance.cosine(a, b)

def closest(vector, compared_to):
    best = -5
    best_idx = -1
#     print(compared_to.shape)
    for i, cmp in enumerate(compared_to):
        c = closeness(vector, cmp)
        if c > best:
            best_idx = i
            best = c
    return best_idx, best

def b_closest(vectors, compared_to):
    return np.array([closest(vector, compared_to)[0] for vector in vectors])

In [ ]:
gc.collect()

In [ ]:
# Get Test And Train Sets
train_X = np.concatenate(train_articles, axis = 0)
train_Y = np.repeat(list(range(len(new_articles))), [train_article.shape[0] for train_article in train_articles], axis = 0)

train_vecs = get_vectors(train_X)

In [ ]:
# # Get Test And Train Sets
# train_X = np.concatenate((
#     train_shirts_navyblue_images,
#     train_shirts_blue_images,
#     train_shirts_black_images,
#     train_shirts_grey_images,
#     train_shirts_green_images,
#     train_shirts_purple_images,
#     train_shirts_white_images,
# ), axis = 0)
# train_Y = np.repeat((0, 1, 2, 3, 4, 5, 6), (
#     train_shirts_navyblue_images.shape[0],
#     train_shirts_blue_images.shape[0],
#     train_shirts_black_images.shape[0],
#     train_shirts_grey_images.shape[0],
#     train_shirts_green_images.shape[0],
#     train_shirts_purple_images.shape[0],
#     train_shirts_white_images.shape[0],
# ), axis = 0)

# train_vecs = get_vectors(train_X)

In [ ]:
# Get Test And Train Sets
test_X = np.concatenate(test_articles, axis = 0)
test_Y = np.repeat(list(range(len(new_articles))), [test_article.shape[0] for test_article in test_articles], axis = 0)

test_vecs = get_vectors(test_X)

In [ ]:
# # Get Test And Train Sets
# test_X = np.concatenate((
#     test_shirts_navyblue_images,
#     test_shirts_blue_images,
#     test_shirts_black_images,
#     test_shirts_grey_images,
#     test_shirts_green_images,
#     test_shirts_purple_images,
#     test_shirts_white_images,
# ), axis = 0)
# test_Y = np.repeat((0, 1, 2, 3, 4, 5, 6), (
#     test_shirts_navyblue_images.shape[0],
#     test_shirts_blue_images.shape[0],
#     test_shirts_black_images.shape[0],
#     test_shirts_grey_images.shape[0],
#     test_shirts_green_images.shape[0],
#     test_shirts_purple_images.shape[0],
#     test_shirts_white_images.shape[0],
# ), axis = 0)

# test_vecs = get_vectors(test_X)

In [ ]:
gc.collect()

In [ ]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
trainx, trainy = shuffle(train_vecs, train_Y)
trainx.shape, trainy.shape
tx, vx, ty, vy = train_test_split(trainx, trainy, test_size = 0.2)
testx, testy = test_vecs.copy(), test_Y.copy()
tx.shape, ty.shape, vx.shape, vy.shape, testx.shape, testy.shape

In [ ]:
scaler = MinMaxScaler()
tx = scaler.fit_transform(tx)
vx = scaler.transform(vx)
testx = scaler.transform(testx)

tx.shape, ty.shape, vx.shape, vy.shape, testx.shape, testy.shape

In [ ]:
EPOCHS = 300
BATCH_SIZE = 100
LEARNING_RATE = 0.00007
NUM_FEATURES = 1000
NUM_CLASSES = NUM_OUTPUTS

In [ ]:
# class MulticlassClassification(nn.Module):
#     def __init__(self, num_feature, num_class):
#         super(MulticlassClassification, self).__init__()
        
#         self.layer_1 = nn.Linear(num_feature, 512)
#         self.layer_2 = nn.Linear(512, 128)
#         self.layer_3 = nn.Linear(128, 64)
#         self.layer_out = nn.Linear(64, num_class) 
        
#         self.relu = nn.ReLU()
#         self.dropout = nn.Dropout(p=0.2)
#         self.batchnorm1 = nn.BatchNorm1d(512)
#         self.batchnorm2 = nn.BatchNorm1d(128)
#         self.batchnorm3 = nn.BatchNorm1d(64)
        
#     def forward(self, x):
#         x = self.layer_1(x)
#         x = self.batchnorm1(x)
#         x = self.relu(x)
        
#         x = self.layer_2(x)
#         x = self.batchnorm2(x)
#         x = self.relu(x)
#         x = self.dropout(x)
        
#         x = self.layer_3(x)
#         x = self.batchnorm3(x)
#         x = self.relu(x)
#         x = self.dropout(x)
        
#         x = self.layer_out(x)
        
#         return x

In [ ]:
device = torch.device("cpu")
print(device)

In [ ]:
# torch_model = MulticlassClassification(num_feature = NUM_FEATURES, num_class=NUM_CLASSES)
# torch_model.to(device)

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(torch_model.parameters(), lr=LEARNING_RATE)
# print(torch_model)

In [ ]:
# def multi_acc(y_pred, y_test):
#     y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
#     _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    
#     correct_pred = (y_pred_tags == y_test).float()
#     acc = correct_pred.sum() / len(correct_pred)
    
#     acc = torch.round(acc) * 100
    
#     return acc

In [ ]:
accuracy_stats = {
    'train': [],
    "val": []
}
loss_stats = {
    'train': [],
    "val": []
}

In [ ]:
# class ClassifierDataset(Dataset):
    
#     def __init__(self, X_data, y_data):
#         self.X_data = X_data
#         self.y_data = y_data
        
#     def __getitem__(self, index):
#         return self.X_data[index], self.y_data[index]
        
#     def __len__ (self):
#         return len(self.X_data)


# train_dataset = ClassifierDataset(torch.from_numpy(tx).float(), torch.from_numpy(ty).long())
# val_dataset = ClassifierDataset(torch.from_numpy(vx).float(), torch.from_numpy(vy).long())
# test_dataset = ClassifierDataset(torch.from_numpy(testx).float(), torch.from_numpy(testy).long())

In [ ]:
# train_loader = DataLoader(dataset=train_dataset,
#                           batch_size=BATCH_SIZE
# )
# val_loader = DataLoader(dataset=val_dataset, batch_size=1)
# test_loader = DataLoader(dataset=test_dataset, batch_size=1)

In [ ]:
# print("Begin training.")
# for e in tqdm(range(1, EPOCHS+1)):
#     # TRAINING
#     train_epoch_loss = 0
#     train_epoch_acc = 0
#     torch_model.train()
#     for X_train_batch, y_train_batch in train_loader:
#         X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
#         optimizer.zero_grad()
        
#         y_train_pred = torch_model(X_train_batch)
        
#         train_loss = criterion(y_train_pred, y_train_batch)
#         train_acc = multi_acc(y_train_pred, y_train_batch)
        
#         train_loss.backward()
#         optimizer.step()
        
#         train_epoch_loss += train_loss.item()
#         train_epoch_acc += train_acc.item()
        
        
#     # VALIDATION    
#     with torch.no_grad():
        
#         val_epoch_loss = 0
#         val_epoch_acc = 0
        
#         torch_model.eval()
#         for X_val_batch, y_val_batch in val_loader:
#             X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            
#             y_val_pred = torch_model(X_val_batch)
                        
#             val_loss = criterion(y_val_pred, y_val_batch)
#             val_acc = multi_acc(y_val_pred, y_val_batch)
            
#             val_epoch_loss += val_loss.item()
#             val_epoch_acc += val_acc.item()
#             loss_stats['train'].append(train_epoch_loss/len(train_loader))
#     loss_stats['val'].append(val_epoch_loss/len(val_loader))
#     accuracy_stats['train'].append(train_epoch_acc/len(train_loader))
#     accuracy_stats['val'].append(val_epoch_acc/len(val_loader))
                              
    
#     print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(val_loader):.5f} | Train Acc: {train_epoch_acc/len(train_loader):.3f}| Val Acc: {val_epoch_acc/len(val_loader):.3f}')

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn import metrics

In [ ]:
abc = AdaBoostClassifier(n_estimators=400, learning_rate=1)
abcm = abc.fit(trainx, trainy)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
preds = abc.predict(train_vecs)
acc = accuracy_score(train_Y, preds)
print(f"In sample Accuracy: {acc}")

print(classification_report(train_Y, preds))

preds = abc.predict(test_vecs)
acc = accuracy_score(test_Y, preds)
print(f"Out of sample Accuracy: {acc}")

print(classification_report(test_Y, preds))

In [51]:
import pickle
pickle.dump(abc, open('ada-vvlarge-964.pickle', 'wb'))

In [ ]:
resnet50_model.save('k.h5')
model_json = resnet50_model.to_json()
with open("kk.json", "w") as json_file:
    json_file.write(model_json)

In [58]:
test_img = load_imgs(['index'])

In [59]:
vec = get_vectors(test_img)

In [60]:
vec.shape

(1, 1000)

In [61]:
preds = abc.predict(vec)

In [62]:
preds

array([1])

In [39]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [40]:
trainx, trainy = shuffle(train_vecs, train_Y)
trainx.shape, trainy.shape
tx, vx, ty, vy = train_test_split(trainx, trainy, test_size = 0.2)
testx, testy = test_vecs.copy(), test_Y.copy()
tx.shape, ty.shape, vx.shape, vy.shape, testx.shape, testy.shape

((3181, 1000), (3181,), (796, 1000), (796,), (1001, 1000), (1001,))

In [41]:
trainx2, trainy2 = shuffle(train_vecs, train_Y)
trainx2.shape, trainy2.shape
tx2, vx2, ty2, vy2 = train_test_split(trainx, trainy, test_size = 0.2)
testx2, testy2 = test_vecs.copy(), test_Y.copy()
tx2.shape, ty2.shape, vx2.shape, vy2.shape, testx2.shape, testy2.shape

((3181, 1000), (3181,), (796, 1000), (796,), (1001, 1000), (1001,))

In [42]:
scaler = MinMaxScaler()
tx2 = scaler.fit_transform(tx2)
vx2 = scaler.transform(vx2)
testx2 = scaler.transform(testx2)

tx2.shape, ty2.shape, vx2.shape, vy2.shape, testx2.shape, testy2.shape

((3181, 1000), (3181,), (796, 1000), (796,), (1001, 1000), (1001,))

In [43]:
EPOCHS = 300
BATCH_SIZE = 100
LEARNING_RATE = 0.00007
NUM_FEATURES = 1000
NUM_CLASSES = NUM_OUTPUTS

In [44]:
class MulticlassClassification2(nn.Module):
    def __init__(self, num_feature, num_class):
        super(MulticlassClassification2, self).__init__()
        
        self.layer_1 = nn.Linear(num_feature, 512)
        self.layer_2 = nn.Linear(512, 128)
        self.layer_3 = nn.Linear(128, 256)
        self.layer_4 = nn.Linear(256, 64)
        self.layer_out = nn.Linear(64, num_class) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)
        self.batchnorm1 = nn.BatchNorm1d(512)
        self.batchnorm2 = nn.BatchNorm1d(128)
        self.batchnorm3 = nn.BatchNorm1d(256)
        self.batchnorm4 = nn.BatchNorm1d(64)
        
    def forward(self, x):
        x = self.layer_1(x)
        x = self.batchnorm1(x)
        x = self.relu(x)
        
        x = self.layer_2(x)
        x = self.batchnorm2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_3(x)
        x = self.batchnorm3(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_4(x)
        x = self.batchnorm4(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_out(x)
        
        return x

In [45]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [46]:
torch_model2 = MulticlassClassification2(num_feature = NUM_FEATURES, num_class=NUM_CLASSES)
torch_model2.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(torch_model2.parameters(), lr=LEARNING_RATE)
print(torch_model2)



def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    
    acc = torch.round(acc) * 100
    
    return acc

accuracy_stats = {
    'train': [],
    "val": []
}
loss_stats = {
    'train': [],
    "val": []
}

MulticlassClassification2(
  (layer_1): Linear(in_features=1000, out_features=512, bias=True)
  (layer_2): Linear(in_features=512, out_features=128, bias=True)
  (layer_3): Linear(in_features=128, out_features=256, bias=True)
  (layer_4): Linear(in_features=256, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=3, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (batchnorm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [47]:
class ClassifierDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


train_dataset = ClassifierDataset(torch.from_numpy(tx).float(), torch.from_numpy(ty).long())
val_dataset = ClassifierDataset(torch.from_numpy(vx).float(), torch.from_numpy(vy).long())
test_dataset = ClassifierDataset(torch.from_numpy(testx).float(), torch.from_numpy(testy).long())

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE)
val_loader = DataLoader(dataset=val_dataset, batch_size=1)
test_loader = DataLoader(dataset=test_dataset, batch_size=1)

train_dataset2 = ClassifierDataset(torch.from_numpy(tx2).float(), torch.from_numpy(ty2).long())
val_dataset2 = ClassifierDataset(torch.from_numpy(vx2).float(), torch.from_numpy(vy2).long())
test_dataset2 = ClassifierDataset(torch.from_numpy(testx2).float(), torch.from_numpy(testy2).long())

train_loader2 = DataLoader(dataset=train_dataset2, batch_size=BATCH_SIZE)
val_loader2 = DataLoader(dataset=val_dataset2, batch_size=1)
test_loader2 = DataLoader(dataset=test_dataset2, batch_size=1)

In [48]:
print("Begin training.")
for e in tqdm(range(1, EPOCHS+1)):
    gc.collect()
    # TRAINING
    train_epoch_loss = 0
    train_epoch_acc = 0
    torch_model2.train()
    for X_train_batch, y_train_batch in train_loader2:
        X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
        optimizer.zero_grad()
        
        y_train_pred = torch_model2(X_train_batch)
        
        train_loss = criterion(y_train_pred, y_train_batch)
        train_acc = multi_acc(y_train_pred, y_train_batch)
        
        train_loss.backward()
        optimizer.step()
        
        train_epoch_loss += train_loss.item()
        train_epoch_acc += train_acc.item()
        
        
    # VALIDATION    
    with torch.no_grad():
        
        val_epoch_loss = 0
        val_epoch_acc = 0
        
        torch_model2.eval()
        for X_val_batch, y_val_batch in val_loader:
            X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            
            y_val_pred = torch_model2(X_val_batch)
                        
            val_loss = criterion(y_val_pred, y_val_batch)
            val_acc = multi_acc(y_val_pred, y_val_batch)
            
            val_epoch_loss += val_loss.item()
            val_epoch_acc += val_acc.item()
            loss_stats['train'].append(train_epoch_loss/len(train_loader))
    loss_stats['val'].append(val_epoch_loss/len(val_loader))
    accuracy_stats['train'].append(train_epoch_acc/len(train_loader))
    accuracy_stats['val'].append(val_epoch_acc/len(val_loader))
                              
    gc.collect()
    print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(val_loader):.5f} | Train Acc: {train_epoch_acc/len(train_loader):.3f}| Val Acc: {val_epoch_acc/len(val_loader):.3f}')

Begin training.


RuntimeError: cuda runtime error (710) : device-side assert triggered at C:/cb/pytorch_1000000000000/work/aten/src\THC/THCTensorMathReduce.cuh:531

In [ ]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [ ]:
trainx, trainy = shuffle(train_vecs, train_Y)
# trainx = np.expand_dims(trainx, axis=1)
trainx.shape, trainy.shape

In [ ]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=1000, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

In [ ]:
kfold = KFold(n_splits=10, shuffle=True)

In [ ]:
classify_model = estimator.fit(trainx, trainy)

In [ ]:
preds = estimator.predict(train_vecs)
acc = accuracy_score(train_Y, preds)
print(f"In sample Accuracy: {acc}")

In [ ]:
preds = estimator.predict(test_vecs)
acc = accuracy_score(test_Y, preds)
print(f"Out of sample Accuracy: {acc}")

In [ ]:
estimator.model.save('keras-vlarge-963.h5')